In [1]:
import gym
import numpy as np
import copy, random, os, subprocess, cv2
import tensorflow as tf
import keras.backend as K
from tensorflow.keras import layers, models, regularizers

/opt/conda/lib/python3.7/site-packages/flatbuffers/compat.py:19: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  import imp
/opt/conda/lib/python3.7/site-packages/keras_preprocessing/image/utils.py:23: DeprecationWarning: NEAREST is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.NEAREST or Dither.NONE instead.
  'nearest': pil_image.NEAREST,
/opt/conda/lib/python3.7/site-packages/keras_preprocessing/image/utils.py:24: DeprecationWarning: BILINEAR is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.BILINEAR instead.
  'bilinear': pil_image.BILINEAR,
/opt/conda/lib/python3.7/site-packages/keras_preprocessing/image/utils.py:25: DeprecationWarning: BICUBIC is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.BICUBIC instead.
  'bicubic': pil_image.BICUBIC,
/opt/conda/lib/python3.7/site-packages/keras_preprocessing/image/utils.py:28: Depreca

In [2]:
MODEL_NAME = 'model10'
BUCKET = 'gs://etsuji-car-racing-v2'
os.environ['BUCKET'] = BUCKET
!gsutil mb -c regional -l us-west1 $BUCKET
!gsutil ls $BUCKET

Creating gs://etsuji-car-racing-v2/...
ServiceException: 409 A Cloud Storage bucket named 'etsuji-car-racing-v2' already exists. Try another name. Bucket names must be globally unique across all Google Cloud projects, including those outside of your organization.
gs://etsuji-car-racing-v2/model04/
gs://etsuji-car-racing-v2/model05/
gs://etsuji-car-racing-v2/model06/
gs://etsuji-car-racing-v2/model07/
gs://etsuji-car-racing-v2/model08/
gs://etsuji-car-racing-v2/model09/
gs://etsuji-car-racing-v2/model10/
gs://etsuji-car-racing-v2/model11/


In [3]:
class ApplySoftMaxWeight(layers.Layer):
    def __init__(self, **kwargs):
        self.filter_shape = None
        super(ApplySoftMaxWeight, self).__init__(**kwargs)

    def build(self, input_shape):
        self.kernel = self.add_weight(name='weights', shape=[input_shape[3]])
        self.filter_shape = input_shape

    def get_config(self):
        config = super().get_config()
        return config
    
    def call(self, inputs, **kwargs):
        return inputs * tf.nn.softmax(self.kernel)
    
# Base model
class QValue:
    def __init__(self):
        self.model = self.build_model()

    def build_model(self):
        cnn_input = layers.Input(shape=(48, 48, 3), name='cnn_input')
        cnn1 = layers.Conv2D(16, (5, 5), padding='same',
                             use_bias=True, activation='relu',
                             name='cnn1')(cnn_input)
        pool1 = layers.MaxPooling2D((2, 2), name='pool1')(cnn1)
        cnn2 = layers.Conv2D(16, (5, 5), padding='same',
                             use_bias=True, activation='relu',
                             name='cnn2')(pool1)        
        pool2 = layers.MaxPooling2D((2, 2), name='pool2')(cnn2)
        weighted_filters = ApplySoftMaxWeight(name='weighted_filters')(pool2)

        cnn_flatten = layers.Flatten(name='flatten')(weighted_filters)
        action_input = layers.Input(shape=(5,), name='action_input')
        combined = layers.concatenate([cnn_flatten, action_input], name='concat')
        hidden1 = layers.Dense(2048, activation='relu', name='dense1')(combined)
        hidden2 = layers.Dense(1024, activation='relu', name='dense2')(hidden1)
        hidden3 = layers.Dense(512, activation='relu', name='dense3')(hidden2)
        q_value = layers.Dense(1, name='output')(hidden3)

        model = models.Model(inputs=[cnn_input, action_input], outputs=q_value)
        model.compile(loss='mse')
        return model

    def get_action(self, state):
        states = []
        actions = []
        for a in range(5):
            states.append(np.array(state))
            action_onehot = np.zeros(5)
            action_onehot[a] = 1
            actions.append(action_onehot)
  
        q_values = self.model.predict([np.array(states), np.array(actions)])
        optimal_action = np.argmax(q_values)
        return optimal_action, q_values[optimal_action][0]

In [4]:
q_value = QValue()
q_value.model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 cnn_input (InputLayer)         [(None, 48, 48, 3)]  0           []                               
                                                                                                  
 cnn1 (Conv2D)                  (None, 48, 48, 16)   1216        ['cnn_input[0][0]']              
                                                                                                  
 pool1 (MaxPooling2D)           (None, 24, 24, 16)   0           ['cnn1[0][0]']                   
                                                                                                  
 cnn2 (Conv2D)                  (None, 24, 24, 16)   6416        ['pool1[0][0]']                  
                                                                                              

2022-08-07 21:16:25.091352: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-08-07 21:16:25.091401: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (tensorflow-2-8-20220801-173936): /proc/driver/nvidia/version does not exist
2022-08-07 21:16:25.092638: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [5]:
def join_frames(o0, o1, o2):
    gray_image0 = cv2.cvtColor(cv2.resize(o0, (48, 48)), cv2.COLOR_RGB2GRAY)
    gray_image1 = cv2.cvtColor(cv2.resize(o1, (48, 48)), cv2.COLOR_RGB2GRAY)
    gray_image2 = cv2.cvtColor(cv2.resize(o2, (48, 48)), cv2.COLOR_RGB2GRAY)
    
    return np.array(
        [gray_image0.transpose(),
         gray_image1.transpose(),
         gray_image2.transpose()]).transpose()

In [6]:
def get_episode(environ, q_value, epsilon):
    episode = []
    o0 = environ.reset()
    o1 = copy.deepcopy(o0)
    o2 = copy.deepcopy(o0)
    total_r = 0

    if epsilon > 0:
        keep_count = 3
    else:
        keep_count = 1

    c = 0
    while True:
        if c % keep_count == 0: # Get new action
            if np.random.random() < epsilon:
                a = np.random.randint(5)
            else:
                a, _ = q_value.get_action(join_frames(o0, o1, o2))
        c += 1
        o_new, r, done, inf = environ.step(a)                
        total_r += r

        # Terminate episode when total reward becomes negative
        if total_r < 0:
            done = 1

        if done:
            # Terminal state is to achive more than 990 or get out of the field.
            if total_r > 990 or r < -99:
                episode.append((join_frames(o0, o1, o2), a, r, None))
            break
        else:
            episode.append((join_frames(o0, o1, o2), a, r, join_frames(o1, o2, o_new)))
        o0, o1, o2 = o1, o2, o_new

    print('epsilon={}, episode length={}, total rewards={}'.format(epsilon, len(episode), total_r))
    return episode, total_r

In [7]:
def train(environ, q_value, epsilon, checkpoint=0):
    gamma = 0.99
        
    if checkpoint > 0:
        filename = 'car-racing-v2-{}-{}.hd5'.format(checkpoint, MODEL_NAME)
        subprocess.run(['gsutil', 'cp', '{}/{}/{}'.format(BUCKET, MODEL_NAME, filename), './'])
        print('load model {}'.format(filename))
        q_value.model = models.load_model(filename)
        os.remove(filename)

    experience = []
    good_experience = []
    best_r = [-100, -100, -100]

    for n in range(checkpoint + 1, checkpoint + 1000):
        print('iteration {}'.format(n))

        total_len = 0
        if n % 3 == 0:
            print('Testing the current performance...')
            episode, total_r = get_episode(environ, q_value, epsilon=0)
            with open('result.txt', 'a') as f:
                f.write('{},{},{},{}\n'.format(n, epsilon, len(episode), total_r))
            filename = 'car-racing-v2-{}-{}.hd5'.format(n, MODEL_NAME)
            q_value.model.save(filename, save_format='h5')
            subprocess.run(['gsutil', '-m', 'cp',
                            '{}'.format(filename), '{}/{}/'.format(BUCKET, MODEL_NAME)])
            os.remove(filename)
            experience += episode
            total_len += len(episode)

        while total_len < 500:
            episode, total_r = get_episode(environ, q_value, epsilon)
            total_len += len(episode)
            experience += episode

            # Keep the top 3 episodes
            if total_r > min(best_r):
                best_r = best_r[1:] + [total_r]
                good_experience += episode
                if len(good_experience) > 999 * 3:
                    good_experience = good_experience[-999 * 3:]

            
        if len(experience) > 999 * 5: # remember last 5 episodes
            experience = experience[-999 * 5:]

        epsilon = (epsilon - 0.2) * 0.99 + 0.2

        print('Training the model...')
        # Use latest episode + past episodes (sampling) + top 3 episode (sampling)
        latest_experience = experience[-total_len:]
        past_experience = experience[:-total_len]
        examples = latest_experience + \
            random.sample(past_experience, min(len(past_experience), 999)) + \
            random.sample(good_experience, min(len(good_experience), 999))
        
        # Show some statistics
        print('experience length={}'.format(len(experience)))
        print('number of examples={}'.format(len(examples)))
        print('best total reward = ', best_r)
        np.random.shuffle(examples)
           
        states, actions, labels = [], [], []
        for state, a, r, state_new in examples:
            states.append(np.array(state))

            action_onehot = np.zeros(5)
            action_onehot[a] = 1
            actions.append(action_onehot)
            
            if state_new is None:   # Terminal state
                q_new = 0
            else:
                _, q_new = q_value.get_action(state_new)
            labels.append(np.array(r + gamma * q_new))

        hist = q_value.model.fit(
            [np.array(states), np.array(actions)], np.array(labels),
            batch_size=50, epochs=10, verbose=0)
        print('loss = {}'.format(hist.history['loss']))

In [8]:
env = gym.make("CarRacing-v2", continuous=False)
q_value = QValue()
q_value.model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 cnn_input (InputLayer)         [(None, 48, 48, 3)]  0           []                               
                                                                                                  
 cnn1 (Conv2D)                  (None, 48, 48, 16)   1216        ['cnn_input[0][0]']              
                                                                                                  
 pool1 (MaxPooling2D)           (None, 24, 24, 16)   0           ['cnn1[0][0]']                   
                                                                                                  
 cnn2 (Conv2D)                  (None, 24, 24, 16)   6416        ['pool1[0][0]']                  
                                                                                            

/opt/conda/lib/python3.7/site-packages/gym/core.py:330: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  "Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future."
/opt/conda/lib/python3.7/site-packages/gym/wrappers/step_api_compatibility.py:40: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  "Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future."


In [ ]:
train(env, q_value, epsilon=1.0, checkpoint=0)

iteration 1
epsilon=1.0, episode length=92, total rewards=-0.06923076923075944
epsilon=1.0, episode length=67, total rewards=-0.02033898305084067
epsilon=1.0, episode length=247, total rewards=-0.06501766784445562
epsilon=1.0, episode length=101, total rewards=-0.09898989898988408
Training the model...
experience length=507
number of examples=913
best total reward =  [-0.06923076923075944, -0.02033898305084067, -0.06501766784445562]
loss = [3356.1015625, 1.6678630113601685, 1.0135606527328491, 0.4260576069355011, 0.30588632822036743, 0.3382064402103424, 0.36367982625961304, 0.23736059665679932, 0.3600166440010071, 0.27549752593040466]
iteration 2
epsilon=0.992, episode length=595, total rewards=-0.02553191489366144
Training the model...
experience length=1102
number of examples=2101
best total reward =  [-0.02033898305084067, -0.06501766784445562, -0.02553191489366144]
loss = [0.5754793286323547, 0.4101273715496063, 0.3794131278991699, 0.4025336802005768, 0.5930966734886169, 0.33036795

Copying file://car-racing-v2-3-model10.hd5 [Content-Type=application/octet-stream]...
/ [1/1 files][ 56.2 MiB/ 56.2 MiB] 100% Done                                    
Operation completed over 1 objects/56.2 MiB.                                     


epsilon=0.9840800000000001, episode length=78, total rewards=-0.08749999999998903
epsilon=0.9840800000000001, episode length=91, total rewards=-0.08145896656534224
epsilon=0.9840800000000001, episode length=151, total rewards=-0.04848484848481968
epsilon=0.9840800000000001, episode length=153, total rewards=-0.06257668711653808
Training the model...
experience length=1646
number of examples=2542
best total reward =  [-0.02553191489366144, -0.04848484848481968, -0.06257668711653808]
loss = [0.3237617313861847, 0.28124645352363586, 0.28037041425704956, 0.27460169792175293, 0.25909319519996643, 0.26945605874061584, 0.24346671998500824, 0.25470221042633057, 0.24108807742595673, 0.24178534746170044]
iteration 4
epsilon=0.9762392000000002, episode length=146, total rewards=-0.04798534798531337
epsilon=0.9762392000000002, episode length=218, total rewards=-0.08181818181815648
epsilon=0.9762392000000002, episode length=94, total rewards=-0.0362776025236535
epsilon=0.9762392000000002, episode l

Copying file://car-racing-v2-6-model10.hd5 [Content-Type=application/octet-stream]...
- [1/1 files][ 56.2 MiB/ 56.2 MiB] 100% Done                                    
Operation completed over 1 objects/56.2 MiB.                                     


Training the model...
experience length=3789
number of examples=2622
best total reward =  [-0.0030075187969604567, -0.006329113924047308, -0.007317073170687055]
loss = [0.2677551209926605, 0.24869705736637115, 0.23781681060791016, 0.23458141088485718, 0.24464690685272217, 0.21639211475849152, 0.2262457311153412, 0.2250456064939499, 0.22089137136936188, 0.21392670273780823]
iteration 7
epsilon=0.9531841195208004, episode length=191, total rewards=-0.09171974522290927
epsilon=0.9531841195208004, episode length=107, total rewards=-0.04731182795696684
epsilon=0.9531841195208004, episode length=250, total rewards=-0.09999999999994924
Training the model...
experience length=4337
number of examples=2546
best total reward =  [-0.0030075187969604567, -0.006329113924047308, -0.007317073170687055]
loss = [0.29807958006858826, 0.2691815197467804, 0.2453012466430664, 0.2409779131412506, 0.2538585960865021, 0.22817566990852356, 0.24041618406772614, 0.21936140954494476, 0.2256382405757904, 0.21613465

...

Training the model...
experience length=4995
number of examples=2997
best total reward =  [39.44223107569623, 53.84615384615471, 49.85590778097862]
loss = [0.8680939078330994, 0.7029879689216614, 0.639191210269928, 0.5544512867927551, 0.5402981042861938, 0.4873291254043579, 0.457379549741745, 0.4680476784706116, 0.4446582794189453, 0.42019355297088623]
iteration 79
epsilon=0.5652877981951332, episode length=999, total rewards=8.108108108107023
Training the model...
experience length=4995
number of examples=2997
best total reward =  [39.44223107569623, 53.84615384615471, 49.85590778097862]
loss = [0.6951237320899963, 0.5626992583274841, 0.5024874210357666, 0.43951940536499023, 0.4251699149608612, 0.38500791788101196, 0.4497198164463043, 0.3242715299129486, 0.3553963601589203, 0.3564733862876892]
iteration 80
epsilon=0.5616349202131818, episode length=684, total rewards=-0.006849315069016287
Training the model...
experience length=4995
number of examples=2682
best total reward =  [39.442

Copying file://car-racing-v2-81-model10.hd5 [Content-Type=application/octet-stream]...
/ [1/1 files][ 56.2 MiB/ 56.2 MiB] 100% Done                                    
Operation completed over 1 objects/56.2 MiB.                                     


Training the model...
experience length=4995
number of examples=2655
best total reward =  [39.44223107569623, 53.84615384615471, 49.85590778097862]
loss = [0.7472199201583862, 0.4990912973880768, 0.49399882555007935, 0.39650148153305054, 0.3961038589477539, 0.3335897624492645, 0.32406753301620483, 0.3430142104625702, 0.30133119225502014, 0.3048293888568878]
iteration 82
epsilon=0.5544383853009396, episode length=817, total rewards=-0.015613382900341949
Training the model...
experience length=4995
number of examples=2815
best total reward =  [39.44223107569623, 53.84615384615471, 49.85590778097862]
loss = [0.671235978603363, 0.5285197496414185, 0.4517378509044647, 0.4198106527328491, 0.5187810063362122, 0.40467777848243713, 0.32497328519821167, 0.3479909598827362, 0.33315861225128174, 0.3979910612106323]
iteration 83
epsilon=0.5508940014479302, episode length=827, total rewards=-0.09323308270758557
Training the model...
experience length=4995
number of examples=2825
best total reward = 

Copying file://car-racing-v2-84-model10.hd5 [Content-Type=application/octet-stream]...
/ [1/1 files][ 56.2 MiB/ 56.2 MiB] 100% Done                                    
Operation completed over 1 objects/56.2 MiB.                                     


Training the model...
experience length=4995
number of examples=2997
best total reward =  [39.44223107569623, 53.84615384615471, 49.85590778097862]
loss = [1.170850157737732, 0.8774429559707642, 0.7583107352256775, 0.700752317905426, 0.5909109115600586, 0.615998387336731, 0.5106540322303772, 0.5424823760986328, 0.47290703654289246, 0.43147191405296326]
iteration 85
epsilon=0.5439112108191164, episode length=773, total rewards=-0.0006191950468174834
Training the model...
experience length=4995
number of examples=2771
best total reward =  [39.44223107569623, 53.84615384615471, 49.85590778097862]
loss = [0.7999150156974792, 0.5659582018852234, 0.5065125226974487, 0.43928417563438416, 0.4087236225605011, 0.40213948488235474, 0.3598531484603882, 0.3908293545246124, 0.3456452786922455, 0.3198377192020416]
iteration 86
epsilon=0.5404720987109253, episode length=748, total rewards=-0.049700598802943946
Training the model...
experience length=4995
number of examples=2746
best total reward =  [3